In [2]:
import pandas as pd

# Read the test data
test_defog = pd.read_csv('test/defog/02ab235146.csv')
# create a new column for the id which is the filename followed by '_' and the Time
test_defog['Id'] = '003f117e14' + '_' + test_defog['Time'].astype(str)
test_defog['Source'] = 'defog'

test_tdcsfog = pd.read_csv('test/tdcsfog/003f117e14.csv')
# create a new column for the id which is '003f117e14' followed by '_' and the Time
test_tdcsfog['Id'] = '003f117e14' + '_' + test_tdcsfog['Time'].astype(str)
test_tdcsfog['Source'] = 'tdcsfog'

# Combine the test data
test_data = pd.concat([test_defog, test_tdcsfog], ignore_index=False)
# set the index to be 'Unnamed: 0'
# test_data['Unnamed: 0'] = test_data.index

In [3]:
def unify_acceleration_units(data):
    g_to_ms2 = 9.81

    data.loc[data['Source'].isin(['defog', 'notype']), ['AccV', 'AccML', 'AccAP']] *= g_to_ms2
    return data

test_data = unify_acceleration_units(test_data)

In [6]:
pro = pd.read_csv('processed.csv')
pro.head()

,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,-9.533939,0.566322,-1.413525,0.0,0.0,0.0
1,-9.536140,0.564137,-1.440621,0.0,0.0,0.0
2,-9.529345,0.561765,-1.429332,0.0,0.0,0.0
3,-9.531239,0.564227,-1.415490,0.0,0.0,0.0
4,-9.540825,0.561854,-1.429471,0.0,0.0,0.0


In [4]:
from autogluon.tabular import TabularPredictor

# Load the saved models
predictor_start_hes = TabularPredictor.load('AutogluonModels/start_hes', 
                                            require_version_match=False,
                                            require_py_version_match=False)
predictor_turn = TabularPredictor.load('AutogluonModels/turn', 
                                       require_version_match=False,
                                       require_py_version_match=False)
predictor_walking = TabularPredictor.load('AutogluonModels/walking', 
                                          require_version_match=False,
                                          require_py_version_match=False)

# Make predictions on the test data for each target event type
labels = ['StartHesitation', 'Turn', 'Walming']
predictors = {
    'StartHesitation': predictor_start_hes,
    'Turn': predictor_turn,
    'Walking': predictor_walking
}

for label in labels:
    print(f"Predicting probabilities for {label}...")
    predictor = predictors[label]
    predictions = predictor.predict((test_data), model='WeightedEnsemble_L2')
    print(predictions)
    print(type(predictions))

# Combine the predictions into a single DataFrame
predictions_df = pd.concat(predictions, axis=1)
predictions_df.columns = labels
print(predictions_df)

# Create a new DataFrame with Id and StartHesitation columns
predictions_df = pd.DataFrame({
    'Id': test_data['Id'],
    'StartHesitation': predictions.values
})

# Save the predictions to a CSV file
predictions_df.to_csv('start_hesitation_predictions.csv', index=False)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found 1 mismatches between original and current metadata:
Found 1 mismatches between original and current metadata:
Found 1 mismatches between original and current metadata:


Predicting probabilities for StartHesitation...


KeyError: "2 required columns are missing from the provided dataset to transform using AutoMLPipelineFeatureGenerator. 2 missing columns: ['Turn', 'Walking'] | 6 available columns: ['Time', 'AccV', 'AccML', 'AccAP', 'Id', 'Source']"